# SFNEBlock Demo

This notebook demonstrates the complete functionality of the KMR SFNEBlock (Sparse Feature Network Ensemble), including:
- Model creation and configuration
- Training and evaluation
- Ensemble learning capabilities
- Performance visualization

## Setup and Imports


In [ ]:
import numpy as np
import plotly.graph_objects as go
import tensorflow as tf
import keras
import warnings
warnings.filterwarnings('ignore')

# Import KMR models and utilities
from kmr.models import SFNEBlock
from kmr.utils import KMRPlotter, KMRDataGenerator

print("✅ All imports successful!")
print(f"TensorFlow version: {tf.__version__}")
print(f"Keras version: {keras.__version__}")


## 1. Generate Synthetic Data

We'll create a classification dataset to demonstrate SFNEBlock's capabilities.


In [ ]:
# Generate synthetic classification data using KMR data generator
X_train, X_test, y_train, y_test = KMRDataGenerator.generate_classification_data(
    n_samples=2000,
    n_features=20,
    n_classes=2,
    noise_level=0.1,
    include_interactions=True,
    include_nonlinear=True
)

print(f"Training data shape: {X_train.shape}")
print(f"Test data shape: {X_test.shape}")
print(f"Class distribution: {np.bincount(y_train)}")


## 2. SFNEBlock Model Creation and Training
 ()

In [ ]:
# Create SFNEBlock model with correct parameters
n_features = X_train.shape[1]
feature_names = [f'feature_{i}' for i in range(n_features)]

model = SFNEBlock(
    feature_names=feature_names,
    hidden_units=[64, 32, 16],
    output_units=1,  # Binary classification
    slow_network_units=32,
    slow_network_layers=2
)

print("✅ SFNEBlock created successfully!")
print(f"Model feature names: {model.feature_names}")
print(f"Model hidden units: {model.hidden_units}")
print(f"Model output units: {model.output_units}")
print(f"Slow network units: {model.slow_network_units}")
print(f"Slow network layers: {model.slow_network_layers}")

# Compile the model
model.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy", "precision", "recall"]
)

print("✅ Model compiled successfully!")


In [ ]:
# Train the model
print("🚀 Starting training...")

history = model.fit(
    X_train, y_train,
    validation_data=(X_test, y_test),
    epochs=30,
    batch_size=64,
    verbose=1
)

print("✅ Training completed!")

# Evaluate the model
test_loss, test_accuracy, test_precision, test_recall = model.evaluate(X_test, y_test, verbose=0)
print(f"\n📊 Test Results:")
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")
print(f"Test Precision: {test_precision:.4f}")
print(f"Test Recall: {test_recall:.4f}")

# Calculate F1 score
f1_score = 2 * (test_precision * test_recall) / (test_precision + test_recall)
print(f"Test F1-Score: {f1_score:.4f}")


In [ ]:
# Visualize training progress using KMR plotting utilities
print("📊 Creating training visualizations...")

fig = KMRPlotter.plot_training_history(
    history,
    metrics=['loss', 'accuracy', 'precision', 'recall'],
    title="SFNEBlock Training Progress",
    height=600
)
fig.show()
print("✅ Training visualizations created successfully!")


In [ ]:
# Make predictions and create comprehensive plots
print("🔍 Making predictions and creating comprehensive visualizations...")

# Make predictions
y_pred_proba = model.predict(X_test)
y_pred = (y_pred_proba > 0.5).astype(int).flatten()

# Create comprehensive classification plots
fig = KMRPlotter.create_comprehensive_plot(
    'classification',
    y_true=y_test,
    y_pred=y_pred,
    y_pred_proba=y_pred_proba.flatten(),
    title="SFNEBlock Classification Results"
)
fig.show()

# Calculate additional metrics
accuracy_metric = keras.metrics.BinaryAccuracy()
precision_metric = keras.metrics.Precision()
recall_metric = keras.metrics.Recall()
f1_metric = keras.metrics.F1Score(average='weighted')

accuracy_metric.update_state(y_test, y_pred)
precision_metric.update_state(y_test, y_pred)
recall_metric.update_state(y_test, y_pred)
f1_metric.update_state(y_test, y_pred)

print(f"\n📊 Final Performance Metrics:")
print(f"Accuracy: {accuracy_metric.result().numpy():.4f}")
print(f"Precision: {precision_metric.result().numpy():.4f}")
print(f"Recall: {recall_metric.result().numpy():.4f}")
print(f"F1-Score: {f1_metric.result().numpy():.4f}")

print("✅ SFNEBlock demo completed successfully!")


## 3. Scenario 2: SFNEBlock With Preprocessing


In [ ]:
# Generate multi-input data for preprocessing
feature_shapes = {
    'feature1': (8,),
    'feature2': (6,),
    'feature3': (6,)
}

X_train_dict, X_test_dict, y_train_prep, y_test_prep = KMRDataGenerator.generate_multi_input_data(
    n_samples=2000,
    feature_shapes=feature_shapes,
    task_type="classification"
)

print(f"Multi-input training data shapes:")
for name, data in X_train_dict.items():
    print(f"  {name}: {data.shape}")
print(f"Target shape: {y_train_prep.shape}")


In [ ]:
# Create preprocessing model
preprocessing_model = KMRDataGenerator.create_preprocessing_model(
    input_shapes=feature_shapes,
    output_dim=32,
    name="classification_preprocessing"
)

print("✅ Preprocessing model created successfully!")
print(f"Preprocessing model input shapes: {preprocessing_model.input_shape}")
print(f"Preprocessing model output shape: {preprocessing_model.output_shape}")


In [ ]:
# Create SFNEBlock with preprocessing
model_with_preprocessing = SFNEBlock(
    feature_names=list(feature_shapes.keys()),
    hidden_units=[64, 32, 16],
    output_units=1,  # Binary classification
    slow_network_units=32,
    slow_network_layers=2,
    preprocessing_model=preprocessing_model
)

print("✅ SFNEBlock (with preprocessing) created successfully!")
print(f"Model feature names: {model_with_preprocessing.feature_names}")
print(f"Model hidden units: {model_with_preprocessing.hidden_units}")
print(f"Model output units: {model_with_preprocessing.output_units}")
print(f"Preprocessing model: {model_with_preprocessing.preprocessing_model is not None}")

# Compile the model
model_with_preprocessing.compile(
    optimizer="adam",
    loss="binary_crossentropy",
    metrics=["accuracy", "precision", "recall"]
)

print("✅ Model compiled successfully!")


In [ ]:
# Train the model with preprocessing
print("🚀 Starting training (with preprocessing)...")

history_with_preprocessing = model_with_preprocessing.fit(
    X_train_dict, y_train_prep,
    validation_data=(X_test_dict, y_test_prep),
    epochs=30,
    batch_size=64,
    verbose=1
)

print("✅ Training completed!")

# Evaluate the model
test_loss_prep, test_accuracy_prep, test_precision_prep, test_recall_prep = model_with_preprocessing.evaluate(X_test_dict, y_test_prep, verbose=0)
print(f"\n📊 Test Results (With Preprocessing):")
print(f"Test Loss: {test_loss_prep:.4f}")
print(f"Test Accuracy: {test_accuracy_prep:.4f}")
print(f"Test Precision: {test_precision_prep:.4f}")
print(f"Test Recall: {test_recall_prep:.4f}")

# Calculate F1 score
f1_score_prep = 2 * (test_precision_prep * test_recall_prep) / (test_precision_prep + test_recall_prep)
print(f"Test F1-Score: {f1_score_prep:.4f}")


In [ ]:
# Create training history comparison
print("📊 Creating training visualizations...")

# Plot training history for both models
fig_no_prep = KMRPlotter.plot_training_history(
    history,
    metrics=['loss', 'accuracy', 'precision', 'recall'],
    title="Training Progress - No Preprocessing",
    height=600
)
fig_no_prep.show()

fig_with_prep = KMRPlotter.plot_training_history(
    history_with_preprocessing,
    metrics=['loss', 'accuracy', 'precision', 'recall'],
    title="Training Progress - With Preprocessing",
    height=600
)
fig_with_prep.show()

print("✅ Training visualizations created successfully!")


In [ ]:
# Make predictions and create comprehensive plots
print("🔍 Making predictions and creating comprehensive visualizations...")

# Predictions without preprocessing
y_pred_proba_no_prep = model.predict(X_test)
y_pred_no_prep = (y_pred_proba_no_prep > 0.5).astype(int).flatten()

# Predictions with preprocessing
y_pred_proba_with_prep = model_with_preprocessing.predict(X_test_dict)
y_pred_with_prep = (y_pred_proba_with_prep > 0.5).astype(int).flatten()

# Create comprehensive classification plots
fig_no_prep = KMRPlotter.create_comprehensive_plot(
    'classification',
    y_true=y_test,
    y_pred=y_pred_no_prep,
    y_pred_proba=y_pred_proba_no_prep.flatten(),
    title="SFNEBlock Classification Results - No Preprocessing"
)
fig_no_prep.show()

fig_with_prep = KMRPlotter.create_comprehensive_plot(
    'classification',
    y_true=y_test_prep,
    y_pred=y_pred_with_prep,
    y_pred_proba=y_pred_proba_with_prep.flatten(),
    title="SFNEBlock Classification Results - With Preprocessing"
)
fig_with_prep.show()

print("✅ Comprehensive visualizations created successfully!")
